# Sessie 8 - Lineair Programmeren

## PuLP installatie

In dit WPO hebben jullie de package PuLP nodig. 

Om PulP (en de andere dependencies) te installeren in je default environment:<br>
- Open je default Terminal of Ananaconda Prompt (indien je Anaconda Navigator gebruikt)
- `pip install pulp`
- open dit notebook

PuLP documentatie: https://www.coin-or.org/PuLP/pulp.html

Andere mogelijke dependencies:
- `pandas`
- `matplotlib`

Indien je een 'Cannot execute cbc ...' error krijgt:
- Op Windows en Linux: `conda install conda-forge::coincbc`
- Op MacOS: `brew install cbc`

In [ ]:
%run visualise.py
%matplotlib inline

import pulp
import pandas as pd

## 1. Inconsistente tijdsallocatie

Het einde van de examens is nabij, en je hebt nog 2 examens af te leggen: Software Engineering en Wetenschappelijk Rekenen.
Je hebt de volgende tijdsallocatie opgesteld die nodig is om met zekerheid te slagen voor alle examens.
            
<table>
  <tr>
    <td><b>Cursus</b></td>
    <td><b>Tijd nodig</b></td>
  </tr>
  <tr>
    <td>SE hoc</td>
    <td>12</td>
  </tr>
  <tr>
    <td>SE wpo</td>
    <td>8</td>
  </tr>
  <tr>
    <td>WR hoc</td>
    <td>28</td>
  </tr>
  <tr>
    <td>WR wpo</td>
    <td>19</td>
  </tr>
  <tr>
    <td>Eten, douchen, slapen</td>
    <td>34</td>
  </tr>
</table>

Het probleem is dat je maar <i>67 uur</i> over hebt om alles in te studeren, waardoor je genoodzaakt bent om overal een beetje tijd weg te nemen.

Neem overal evenveel tijd weg, behalve voor:

* <i>Eten, douchen, slapen</i>: Deze taken zijn minder belangrijk tijdens de examens, dus je kan 2 keer zoveel tijd wegnemen als bij de andere.
* <i>WR hoc</i>: Je hebt je tijd nodig voor dit deel, dus neem geen tijd weg.

Zoek nu een nieuwe tijdsallocatie die de deadline respecteert.

In [ ]:
### 1. Problem ###
prob = pulp.LpProblem("Time_allocation", pulp.LpMinimize)

### 2. Variables ###
## Positive hours
SEhoc = pulp.LpVariable("SEhoc", cat="Continuous", lowBound=0)   # Create a continuous variable SEhoc >= 0 
SEwpo = pulp.LpVariable("SEwpo", cat="Continuous", lowBound=0)   # Create a continuous variable SEwpo >= 0 
WRhoc = pulp.LpVariable("WRhoc", cat="Continuous", lowBound=0)   # Create a continuous variable WRhoc >= 0 
WRwpo = pulp.LpVariable("WRwpo", cat="Continuous", lowBound=0)   # Create a continuous variable WRwpo >= 0 
Misc = pulp.LpVariable("Misc", cat="Continuous", lowBound=0)   # Create a continuous variable Misc >= 0 
Residue = pulp.LpVariable("Residue", cat="Continuous", lowBound=0)   # Create a continuous variable Residue >= 0 

In [ ]:
### 3. Objective ###
prob += Residue
  
### 4. Constraints ###
## Time necessary per course (with residuals)
prob += SEhoc == 12 - Residue
prob += SEwpo == 8 - Residue
prob += WRhoc == 28
prob += WRwpo == 19 - Residue
prob += Misc == 34 - 2*Residue
## Deadline
prob += SEhoc + SEwpo + WRhoc + WRwpo + Misc <= 67

In [ ]:
# Display the problem
print(prob) 
    
solver = pulp.getSolver('COIN_CMD')
status = prob.solve(solver)    # Solver 

In [ ]:
print("======> Solver Status", pulp.LpStatus[status])   # The solution status 
print("====> Following solution was found:")
print(f"Software Engineering HOC: {pulp.value(SEhoc)}")
print(f"Software Engineering WPO: {pulp.value(SEwpo)}")
print(f"Wetenschappelijk Rekenen HOC: {pulp.value(WRhoc)}")
print(f"Wetenschappelijk Rekenen WPO: {pulp.value(WRwpo)}")
print(f"Eten, douchen, slapen: {pulp.value(Misc)}")
print(f"===> Minimized residue: {pulp.value(prob.objective)}")

Alternatief kan je ook de oplossing vereenvoudigen op de volgende manier.

In [ ]:
### 1. Problem ###
prob = pulp.LpProblem("Time_allocation", pulp.LpMinimize)

### 2. Variables ###
## X = amount of hours removed
X = pulp.LpVariable("X", cat="Continuous", lowBound=0)
  
### 3. Constraints ###
prob += (12-X) + (8-X) + 28 + (19-X) + (34-2*X) <= 67 # dus 5*X >= 34

### 4. Objective ###
# not necessary for this exercise
# but it is required for PuLP to set the value of the prob.objective
prob += X

# Display the problem
print(prob) 
    
solver = pulp.getSolver('COIN_CMD')
status = prob.solve(solver)    # Solver 

print(f"Hours removed from each (2X for eating, drinking etc.): {pulp.value(X)}")
print(f"===> Objective: {pulp.value(prob.objective)}")

## 2. Mixed products

<i>The ABC Drug Company makes two types of liquid painkiller that have brand names Relieve (R) and Ease (E) and contain different mixtures of three basic drugs, A, B and C, produced by the company. Each bottle of R requires $\frac{7}{9}$ units of drug A, $\frac{1}{2}$ units of drug B, and $\frac{3}{4}$ units of drug C. Each bottle of E requires $\frac{4}{9}$ units of drug A, $\frac{5}{2}$ units of drug B, and $\frac{1}{4}$ units of drug C. The company is able to produce each day only 5 units of drug A, 7 units of drug B, and 9 units of C. Moreover, Food and Drug Administration regulations stipulate that the number of bottles of R manufactured cannot exceed twice the number of bottles of E. The profit margin for each bottle of E and R is 7 euros and 3 euros, respectively. Maximize the owner's profit.</i>

Analyseer elke zin apart en identificeer: (i) wat proberen te minimaliseren/maximaliseren, (ii) de variabelen en (iii) de constraints.

The ABC Drug Company makes two types of liquid painkiller that have brand names Relieve (R) and Ease (E). ;; variable declaration

They contain different mixtures of three basic drugs, A, B and C, produced by the company. 

Each bottle of R requires
- $\frac{7}{9}$ units of drug A, 
- $\frac{1}{2}$ units of drug B, and
- $\frac{3}{4}$ units of drug C. 

Each bottle of E requires 
- $\frac{4}{9}$ units of drug A, 
- $\frac{5}{2}$ units of drug B, and 
- $\frac{1}{4}$ units of drug C. 

The company is able to produce each day only 5 units of drug A, 7 units of drug B, and 9 units of C. ;; constraint 1

Moreover, Food and Drug Administration regulations stipulate that the number of bottles of R manufactured cannot exceed twice the number of bottles of E. ;; constraint 2

The profit margin for each bottle of E and R is 7 euros and 3 euros, respectively. ;; objective function

Maximize the owner's profit. ;; objective declaration

In [ ]:
### 1. Problem definition ###
prob = pulp.LpProblem("mixed_products", pulp.LpMaximize)

### 2. Variable declaration ###
## Positive integer number of products
R = pulp.LpVariable("R", cat="Integer", lowBound=0)   # Create an integer variable R >= 0 
E = pulp.LpVariable("E", cat="Integer", lowBound=0)   # Create an integer variable E >= 0 

In [ ]:
### 3. Objective ###
prob += 3*R + 7*E # profit margins
prob += R <= 2*E
  
### 4. Constraints ###
## Recipes for products (R & E should appear together in one constraint, 
##   otherwise you can use the same amount units in multiple bottles)
prob += 7/9*R + 4/9*E <= 5 # A
prob += 1/2*R + 5/2*E <= 7 # B
prob += 3/4*R + 1/4*E <= 9 # C
## Food and Drugs Administration rule
prob += R <= 2*E

In [ ]:
# Display the problem 
print(prob)
    
solver = pulp.getSolver('COIN_CMD')
status = prob.solve(solver)    # Solver 
print(pulp.LpStatus[status])   # The solution status 
  
# Printing the final solution 
print(pulp.value(R), pulp.value(E), pulp.value(prob.objective))

## 3. Job shop scheduling problemen


Operationeel onderzoek (operations research) richt zich op de ontwikkeling en toepassing van analytische methodes om beslissingsprocessen te verbeteren en te optimaliseren.

In deze sessie onderzoeken we een job-shop scheduling probleem. De job shop bestaat uit een set van machines. Een job bestaat uit een set van taken. \
Elke machine kan een bepaalde set van taken uitvoeren. De tijd om een bepaalde taak uit te voeren op een bepaalde machine is gekend. \
Het doel is om de jobs zo snel mogelijk uit te voeren door de jobs op een efficiënte manier in te plannen op de machines. \
De totale tijd om alle jobs uit te voeren noemt men de <em>makespan</em>.

Vandaag is job-shop scheduling een belangrijke tool voor fabrikanten die de efficiëntie van hun productieprocessen willen verhogen en de verspilling binnen hun productieprocessen wensen te elimineren.

### De printing press

Deze oefening komt van Christelle Gueret, Christian Prins, Marc Sevaux, "Applications of Optimization with Xpress-MP," Dash Optimization, 2000. 

In dit voorbeeld zijn er drie producten ($Paper_{1}, Paper_{2}, Paper_{3}$). Elk product stelt een job voor dat bestaat uit 1 of meer deeltaken. \
In dit probleem moeten de papers door een of meerdere drukpersen. Het volgende diagram beschrijft de volgorde in het productieproces van elk product. \
Bijvoorbeeld $Paper_{3}$ gaat eerst door de gele machine. Daarna dient het resultaat van deze taak als input voor de blauwe machine. Ten laatste gaat de paper door de groene machine.

![Drag Racing](ex3-diagram.jpeg)

De volgende tabel beschrijft de tijd vereist om een gegeven job uit te voeren op een gegeven machine.

| Machine | Color | Paper 1 | Paper 2 | Paper 3| 
| :--- | :----:   | :----: | :----: |  ---: |
| 1    | Green    | -  | 10 | 17|
| 2    | Blue     | 45 | 20 | 12|
| 3    | Yellow   | 10 | 34 | 28|

Formuleer een model om de makespan van dit probleem te minimaliseren. Let op het feit dat niet alle papers alle machines vereisen.

In [ ]:
# amount of machines and jobs
machines = 3
jobs = 3

# sequence of tasks for a job
machine_sequence = [[1, 2],    # paper 1: Blue => Yellow
                    [0, 1, 2], # paper 2: Green => Blue => Yellow
                    [2, 1, 0]] # paper 3: Yellow => Blue => Green
                     
# duration of a task (a pair of a job j and machine m)
dur = [[None, 45, 10],
        [10, 20, 34],
        [17, 12, 28]]

In [ ]:
### 1. Model definition ###
model = pulp.LpProblem(name="printing_press", sense=pulp.LpMinimize)

### 2. Helper variables ###
# all job-machine pairs 
valid_starts = []
for j in range(jobs):
    for m in machine_sequence[j]:
        valid_starts.append((j, m))

# all job-job pairs that compete for the same machine
jjm = []
for j1 in range(jobs):
    for j2 in range(jobs):
        for m in range(machines):
            if j1 != j2 and \
               (j1, m) in valid_starts and \
               (j2, m) in valid_starts:
                jjm.append((j1, j2, m))

# choose Big-M as the horizon of the makespan (i.e., if all tasks are ran sequentially)
# big-m reformulation: https://optimization.cbe.cornell.edu/index.php?title=Disjunctive_inequalities
# Niet te kennen voor examen, wel nodig voor dit probleem omdat pulp geen disjunctive constraints ondersteunt)
M = 0
for j in range(jobs):
    for m in range(machines):
        if dur[j][m] is not None:
            M += dur[j][m]

### 3. Variables ###
# 1. makespan (our objective)
obj = pulp.LpVariable("makespan", lowBound=0, cat='Integer')
# 2. start[j, m] - start time for job j on machine m
start = pulp.LpVariable.dicts("start_time", indexs=valid_starts, lowBound=0, cat='Integer')
# 3. binary[j1, j2, m]
#   whether job j1 precedes j2 on machine m
#   enables us to express disjunctive constraints    
binary = pulp.LpVariable.dicts("precedence", indexs=jjm, cat='Binary')

# put the objective into the model
model += obj 

Allereerst beschrijven we de verschillende constraints van dit probleem:
- Geen enkele taak kan worden gestart voordat de vorige taak is voltooid.
- Een machine kan slechts aan één taak tegelijk werken.
- Eenmaal een taak gestart is, moet die worden voltooid.

In [ ]:
### 4. Constraints ###

# constraint 1: the makespan
for j, m in valid_starts:
    # constrain the makespan ifo the start and duration times of all j,m pairs.
    model += obj >= start[j, m] + dur[j][m]

# constraint 2: enforce machine sequence
for j in range(jobs):
    for m_idx in range(1, len(machine_sequence[j])):
        curr_m = machine_sequence[j][m_idx]
        prev_m = machine_sequence[j][m_idx - 1]
        model += start[j, curr_m] >= start[j, prev_m] + dur[j][prev_m]

# constraint 3: only one active job at a time
for j1, j2, m in jjm:
    # disjunction with Big-M reformulation (niet te kennen voor examen)
    # use binary variable to decide if a job j1 ends before j2 when binary[j1, j2, m] == 1, start[j2, m] >= end[j1, m]
    # or vice-versa binary[j1, j2, m] == 0, start[j1, m] >= end[j2, m]
    # we use M to make the right-hand side of the inactive constraint so large that it becomes irrelevant
    #   if binary[j1, j2, m] == True, then the second constraint becomes irrelevant
    #   if binary[j1, j2, m] == False, then the first constraint becomes irrelevant
    model += start[j2, m] >= start[j1, m] + dur[j1][m] - (1-binary[j1, j2, m])*M
    model += start[j1, m] >= start[j2, m] + dur[j2][m] - (  binary[j1, j2, m])*M
    model += binary[j1, j2, m] + binary[j2, j1, m] == 1

Om de logische disjunctie uit te drukken kan men een grote M gebruiken samen met een binaire variabele constraints aan of uit te zetten.

In een scheduling probleem kan je bijvoorbeeld een binaire variabele `binary` gebruiken om te beslissen of een job `j1` eerder eindigt dan een andere job `j2` als `binary=1` (of omgekeerd als `binary=0`). 
We gebruiken M om het rechterlid van de inactieve constraint zo groot te maken zodat het irrelevant wordt. We kiezen hierbij de waarde van M groot genoeg. In dit geval kiezen we M als de makespan indien je alle taken sequentieel zou uitvoeren.

In [ ]:
solver = pulp.getSolver('COIN_CMD')
status = model.solve(solver)

print(f"======> Solver Status: {pulp.LpStatus[status]}")
print(f"=====>  Minimal makespan: {model.objective.value()}")

#### Visualising the job shop

Ten laatste kunnen de gevonden schedule visualiseren aan de hand van een Gantt chart. De functie 'format_data_for_visualization' verwacht de variabelen 'start' en de durations 'dur'.

In [ ]:
results = format_data_for_visualisation(start, dur)

schedule = pd.DataFrame(results)
print('\nSchedule by Job')
print(schedule.sort_values(by=['Job','Start']).set_index(['Job', 'Machine']))

print('\nSchedule by Machine')
print(schedule.sort_values(by=['Machine','Start']).set_index(['Machine', 'Job']))

visualize(results)